In [7]:
import json
  
with open('./mens.json') as f:
    d = json.load(f)
    print(d)

[{"id": "b'WdfFXZW/dvMLdhtMZs3ihVDCRHI='", "ratings": {"Overall Rating": "", "Riding Style": "Freestyle", "Riding Level": "Intermediate - Expert", "Fits Boot size (US)": "8-10", "Manufactured in": "Tunisia by Nidecker", "Shape": "True Twin", "Camber Profile": "Hybrid Camber", "Stance": "Centered", "Approx. Weight": "Feels Normal", "Powder": "Average", "Turning": "Experience", "Carving": "Average", "Speed": "Average", "Uneven": "Terrain", "Switch": "Great", "Jumps": "Good", "Jibbing": "Great", "Pipe": "Good", "On": "Snow", "Turn": "Initiation", "Skidded": "Turns", "Flex": "Medium/Soft", "Buttering": "Easy", "Edge": "Hold"}, "meta_data": {"name": "alloy-b-bomb-gt-2020-snowboard-review", "url": "https://thegoodride.com/snowboard-reviews/alloy-b-bomb-gt-2020-snowboard-review", "price": "$0", "image_url": "https://thegoodride.com/assets/lg-gallery/alloy/snowboards/b-bomb-gt/alloy-b-bomb-gt.jpg"}}, {"id": "b'WdfFXZW/dvMLdhtMZs3ihVDCRHI='", "ratings": {"Overall Rating": "", "Riding Style": "F

In [13]:
import ast
import pandas as pd
from scipy.spatial.distance import squareform, pdist

In [13]:
d = ast.literal_eval(d)

In [14]:
d[0]

{'id': "b'WdfFXZW/dvMLdhtMZs3ihVDCRHI='",
 'ratings': {'Overall Rating': '',
  'Riding Style': 'Freestyle',
  'Riding Level': 'Intermediate - Expert',
  'Fits Boot size (US)': '8-10',
  'Manufactured in': 'Tunisia by Nidecker',
  'Shape': 'True Twin',
  'Camber Profile': 'Hybrid Camber',
  'Stance': 'Centered',
  'Approx. Weight': 'Feels Normal',
  'Powder': 'Average',
  'Turning': 'Experience',
  'Carving': 'Average',
  'Speed': 'Average',
  'Uneven': 'Terrain',
  'Switch': 'Great',
  'Jumps': 'Good',
  'Jibbing': 'Great',
  'Pipe': 'Good',
  'On': 'Snow',
  'Turn': 'Initiation',
  'Skidded': 'Turns',
  'Flex': 'Medium/Soft',
  'Buttering': 'Easy',
  'Edge': 'Hold'},
 'meta_data': {'name': 'alloy-b-bomb-gt-2020-snowboard-review',
  'url': 'https://thegoodride.com/snowboard-reviews/alloy-b-bomb-gt-2020-snowboard-review',
  'price': '$0',
  'image_url': 'https://thegoodride.com/assets/lg-gallery/alloy/snowboards/b-bomb-gt/alloy-b-bomb-gt.jpg'}}

In [32]:
df = pd.json_normalize(d)
df.shape

(2, 29)

In [18]:
sel_cols = ['ratings.Riding Style',
       'ratings.Riding Level', 'ratings.Fits Boot size (US)',
       'ratings.Manufactured in', 'ratings.Shape', 'ratings.Camber Profile',
       'ratings.Stance', 'ratings.Approx. Weight', 'ratings.Powder',
       'ratings.Turning', 'ratings.Carving', 'ratings.Speed', 'ratings.Uneven',
       'ratings.Switch', 'ratings.Jumps', 'ratings.Jibbing', 'ratings.Pipe',
       'ratings.On', 'ratings.Turn', 'ratings.Skidded', 'ratings.Flex',
       'ratings.Buttering', 'ratings.Edge']

meta_cols =

In [20]:
df = pd.get_dummies(df, columns=sel_cols)
df.shape

(2, 29)

In [37]:
def distance_function(r, algo='jaccard'):
    dists = pdist(r, algo)
    
    return pd.DataFrame(squareform(dists))

In [ ]:
df_dist = distance_function(df_final_dummies.drop(labels=meta_cols, axis=1).values)